In [35]:
import os

from language_table.environments import blocks
from language_table.environments import language_table
from language_table.environments.rewards import block2block

from matplotlib import pyplot as plt
import mediapy
import numpy as np
import tensorflow_datasets as tfds
from icecream import ic

np.set_printoptions(precision=3)

In [36]:
# tf dataset handling
import tensorflow as tf

dataset = tf.data.Dataset.range(10)

for i in dataset:
    print(i)

tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)
tf.Tensor(5, shape=(), dtype=int64)
tf.Tensor(6, shape=(), dtype=int64)
tf.Tensor(7, shape=(), dtype=int64)
tf.Tensor(8, shape=(), dtype=int64)
tf.Tensor(9, shape=(), dtype=int64)


2024-05-31 01:25:01.713274: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [37]:
dataset = tf.data.Dataset.from_tensor_slices({"a": ([1, 2], [3, 4]), "b": [5, 6]})
list(dataset.as_numpy_iterator())

2024-05-31 01:25:01.736558: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


[{'a': (1, 3), 'b': 5}, {'a': (2, 4), 'b': 6}]

In [38]:
DATASET_PATH = "/data/language_table"
builder = tfds.builder_from_directory(DATASET_PATH)
episode_ds = builder.as_dataset(split="train")
print(episode_ds.element_spec)

{'episode_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'steps': DatasetSpec({'action': TensorSpec(shape=(2,), dtype=tf.float32, name=None), 'is_first': TensorSpec(shape=(), dtype=tf.bool, name=None), 'is_last': TensorSpec(shape=(), dtype=tf.bool, name=None), 'is_terminal': TensorSpec(shape=(), dtype=tf.bool, name=None), 'observation': {'effector_target_translation': TensorSpec(shape=(2,), dtype=tf.float32, name=None), 'effector_translation': TensorSpec(shape=(2,), dtype=tf.float32, name=None), 'instruction': TensorSpec(shape=(512,), dtype=tf.int32, name=None), 'rgb': TensorSpec(shape=(360, 640, 3), dtype=tf.uint8, name=None)}, 'reward': TensorSpec(shape=(), dtype=tf.float32, name=None)}, TensorShape([]))}


In [39]:
def decode_inst(inst):
    """Utility to decode encoded language instruction"""
    return bytes(inst[np.where(inst != 0)].tolist()).decode("utf-8")

In [44]:
def show_episode(episode):
    """Utility to visualize an episode"""
    print("Episode id:", episode["episode_id"].numpy())
    frames = []
    actions = []
    for step in episode["steps"].as_numpy_iterator():
        frames.append(step["observation"]["rgb"])
        actions.append(step["action"])

    print("step numbers: ", len(frames))
    mediapy.show_video(
        frames, title=decode_inst(step["observation"]["instruction"]), fps=5
    )
    # draw the action sequence
    actions = np.array(actions).T
    print("action sequence:")
    print(actions)

In [48]:
# episodes_iter = iter(episode_ds.take(10))

sample_episodes = tf.data.Dataset.sample_from_datasets(episode_ds)
episodes_iter = iter(sample_episodes)

for episode in episodes_iter:
    show_episode(episode)

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x766c7e94ee90>>
Traceback (most recent call last):
  File "/home/j2y/envs/tiny/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 790, in _clean_thread_parent_frames
    active_threads = {thread.ident for thread in threading.enumerate()}
                                                 ^^^^^^^^^^^^^^^^^^^^^
  File "/home/j2y/.pyenv/versions/3.11.9/lib/python3.11/threading.py", line 1509, in enumerate
    with _active_limbo_lock:
KeyboardInterrupt: 
